In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms

class ClassicCNN(nn.Module):
    def __init__(self):
        super(ClassicCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.bn1(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = ClassicCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')


cuda
Epoca [1/5], Loss: 0.1534, Accuracy: 95.39%
Epoca [2/5], Loss: 0.0486, Accuracy: 98.46%
Epoca [3/5], Loss: 0.0267, Accuracy: 99.07%
Epoca [4/5], Loss: 0.0184, Accuracy: 99.39%
Epoca [5/5], Loss: 0.0149, Accuracy: 99.49%
Accuracy sui test: 97.79%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms
import numpy as np


def to_frequency_domain(x):
    return torch.fft.fft2(x)

def to_time_domain(x):
    return torch.fft.ifft2(x).real

class FrequencyConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(FrequencyConv, self).__init__()
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.weights = nn.Parameter(torch.randn(1, out_channels, in_channels, kernel_size, kernel_size, dtype=torch.cfloat))

    def forward(self, x):
        batch_size, _, H, W = x.shape
        
        kernel_padded = torch.zeros((batch_size, self.out_channels, self.in_channels, H, W), dtype=torch.cfloat, device=x.device)
        kernel_padded[:, :, :, :self.kernel_size, :self.kernel_size] = self.weights 
        kernel_freq = to_frequency_domain(kernel_padded)
        
        out_freq = x.unsqueeze(1) * kernel_freq
        return out_freq.sum(dim=2)
    
class CEMNet(nn.Module):
    def __init__(self):
        super(CEMNet, self).__init__()
        self.conv1 = FrequencyConv(1, 8, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

        
    
    def forward(self, x):
        x = self.conv1(x)
        x = torch.abs(x)
        x = self.bn1(x)
        x= torch.fft.ifft2(x).real
        x = x[:, :, :28, :28]
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


ciao=5

class FFTTransform:
    def __call__(self, img):
        img_tensor = transforms.functional.to_tensor(img)

        fft_result = torch.fft.fft2(img_tensor)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

saved_data = torch.load('./transformed_mnist.pt',weights_only=False)

train_images = saved_data['train_images']
train_labels = saved_data['train_labels']
test_images = saved_data['test_images']
test_labels = saved_data['test_labels']

trainset = torch.utils.data.TensorDataset(train_images, train_labels)
testset = torch.utils.data.TensorDataset(test_images, test_labels)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print("Dataset loaded successfully.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CEMNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')

Dataset loaded successfully.
cuda
Epoca [1/5], Loss: 0.4819, Accuracy: 86.50%
Epoca [2/5], Loss: 0.3110, Accuracy: 91.24%
Epoca [3/5], Loss: 0.2919, Accuracy: 91.80%
Epoca [4/5], Loss: 0.2837, Accuracy: 92.07%
Epoca [5/5], Loss: 0.2774, Accuracy: 92.20%
Accuracy sui test: 91.89%


**CIFRA10**

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms

class ClassicCNN(nn.Module):
    def __init__(self):
        super(ClassicCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.bn1(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = ClassicCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
cuda
Epoca [1/5], Loss: 1.5726, Accuracy: 45.02%
Epoca [2/5], Loss: 1.1274, Accuracy: 60.38%
Epoca [3/5], Loss: 0.8387, Accuracy: 70.71%
Epoca [4/5], Loss: 0.6041, Accuracy: 78.89%
Epoca [5/5], Loss: 0.4268, Accuracy: 85.25%
Accuracy sui test: 53.15%


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms
import numpy as np


def to_frequency_domain(x):
    return torch.fft.fft2(x)

def to_time_domain(x):
    return torch.fft.ifft2(x).real

class FrequencyConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(FrequencyConv, self).__init__()
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.weights = nn.Parameter(torch.randn(1, out_channels, in_channels, kernel_size, kernel_size, dtype=torch.cfloat))

    def forward(self, x):
        batch_size, _, H, W = x.shape
        
        kernel_padded = torch.zeros((batch_size, self.out_channels, self.in_channels, H, W), dtype=torch.cfloat, device=x.device)
        kernel_padded[:, :, :, :self.kernel_size, :self.kernel_size] = self.weights 
        kernel_freq = to_frequency_domain(kernel_padded)
        
        out_freq = x.unsqueeze(1) * kernel_freq
        return out_freq.sum(dim=2)
    
class CEMNet(nn.Module):
    def __init__(self):
        super(CEMNet, self).__init__()
        self.conv1 = FrequencyConv(1, 8, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 10)

        
    
    def forward(self, x):
        x = self.conv1(x)
        x = torch.abs(x)
        x = self.bn1(x)
        x= torch.fft.ifft2(x).real
        x = x[:, :, :32, :32]
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x



class FFTTransform:
    def __call__(self, img):
        img_tensor = transforms.functional.to_tensor(img)

        fft_result = torch.fft.fft2(img_tensor)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

saved_data = torch.load('./transformed_cifar10.pt',weights_only=False)

train_images = saved_data['train_images']
train_labels = saved_data['train_labels']
test_images = saved_data['test_images']
test_labels = saved_data['test_labels']

trainset = torch.utils.data.TensorDataset(train_images, train_labels)
testset = torch.utils.data.TensorDataset(test_images, test_labels)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print("Dataset loaded successfully.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CEMNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')

Dataset loaded successfully.
cuda
Epoca [1/5], Loss: 1.9848, Accuracy: 28.73%
Epoca [2/5], Loss: 1.8254, Accuracy: 36.23%
Epoca [3/5], Loss: 1.7875, Accuracy: 37.61%
Epoca [4/5], Loss: 1.7659, Accuracy: 38.63%
Epoca [5/5], Loss: 1.7516, Accuracy: 39.18%
Accuracy sui test: 38.45%
